# Numpy & Pandas

## Numpy and Arrays

Numpy is a powerful numeric library that is essential for anyone analyzing data with Python. Numpy is a huge package that can support a multitude of tasks. Numpy is also inextricably linked to SciPy, a powerful library for scientific computing with capabilities for fitting, linear algebra, machine learning, etc. Here we are just going to cover some of the basics of numpy, but I encourage you to check out the numpy documentation pages (https://numpy.org/doc/stable/) to get an idea of the variety of things you can do.

Arrays are a data type which is fundamental to Numpy. In some ways Numpy arrays are like Python lists:
    - both are used for storing data/objects
    - both are mutable
    - items can be extracted from both using indexing and slicing
    - both can be iterated over

However there are key aspects of arrays that make them very different:
    - most operators act on the elements of an array instead of the array as a whole
    - arrays can only hold data of a single type
    - arrays can efficiently store large amounts of data in memory


In [ ]:
import numpy as np

# create some sample lists
xlist = [1, 2, 3, 4]
ylist = [1, 4, 9, 16]

# create some sample arrays
x = np.array([1, 2, 3, 4])
y = np.array([1, 4, 9, 16])

First let's checkout the different behaviors between lists and arrays

In [ ]:
print(xlist * 4)

print(x * 4)

print(x / 4)

print(xlist / 4)

Notice how the list was repeated 4 times, whereas each element of the array was multiplied by 4 and the result ended up being the same length.

Division works element-wise for arrays, but division is not defined and produces an error when performed on a list.

## Iterating, indexing, and slicing

Iterating over a 1D array looks just like iterating over a list

In [ ]:
for val in xlist:
    print(val)

for val in x:
    print(val)

Iterating an N-dimensional array will iterate over slices along the first dimension.

In [ ]:
y = np.zeros((5, 5))

for val in y:
    print(val)

print()
# you could accomplish the same thing like this (but you probably shouldn't)
for i in range(y.shape[0]):
    val = y[i, :]
    print(val)

We can also select subsets of the array using conditionals.

In [ ]:
xs = x[x < 2]
xs

## Pandas Tables

Pandas is a powerful data analysis package that provides tools for manipulating tabular data. This is particularly useful in many astronomical applications, such as spectroscopy, and time-series. Data is organized into rows and columns where the columns are named and recalled using arbitrary Python objects (strings are the most convenient). This is in contrast to Numpy arrays where columns can only be accessed using integer indices (however, see record arrays https://docs.scipy.org/doc/numpy-1.10.1/user/basics.rec.html).

Sorting, querying, merging, and aggregation are some of the most useful Pandas features, but this tutorial will only scratch the surface. See https://pandas.pydata.org/docs/ for the full documentation.


Pandas is most useful for dealing with heterogeneous and/or large datasets, when merging or complex queries are needed, or if you have metadata associated with columns (e.g. strings as labels).

The basic units/objects in Pandas are the Series and DataFrame objects.

In [ ]:
import pandas as pd

# Lets create a sample Series object
x = [1.0, 2.0, 4.4, 4.5, 8.8, 9.1, 8.7, 2.3, 2.4, 3.1, 5.9]
s = pd.Series(x)

print(s)

We populated a Series starting from a list of floating point numbers. Notice that two columns are printed in the output. Every entry in a Series has a corresponding integer index, generally these indices are created automatically. The data type of the series is printed below the Series itself. Series objects can only store data of a single type, but any data type can be stored.

A Series is like a single column of data in a table. A DataFrame is the Pandas object that represents a full table. Each column in the table is a Series.

There are several ways to construct a Pandas DataFrame, including from Numpy arrays, Python dictionaries, a list of Series objects, reading from a CSV, reading from a URL, etc.

Let's first construct a single-column DataFrame from our series `s`.

In [ ]:
df = pd.DataFrame(s, columns=['sample'])
df

Jupyter has special support for displaying DataFrames, simply typing the variable name of a DataFrame at the end of the cell will present a nicely formatted view of the table.

Let's add some more columns to our DataFrame.

In [ ]:
df['sample_base'] = df['sample'] // 1
df['sample_plus1'] = df['sample'] + 1
df['sample_squared'] = df['sample']**2
df

Notice that we can access the values in a column using two different syntax.

Now sort by the `sample_squared` column

In [ ]:
df = df.sort_values(by='sample_squared')
df

Notice that the indices were re-ordered as well. The indices retain information about the original ordering.

We can also select subsets of the data using conditionals similar to Numpy arrays.

In [ ]:
q1 = df[df['sample'] <= 4]
q1

The `.groupby` method is used to create Pandas `DataFrameGroupBy` object which can be used to calculate statistics within the groups.

In [ ]:
# groups that share a common sample_base field
g = df.groupby('sample_base')

# count number of rows within each group
print(g.count())

We can also merge DataFrames together using a common column.

Lets create a second DataFrame from the same original list of numbers and calculate the `sample_base` field again. We will also calculate a new column called `sample_sqrt`

In [ ]:
df2 = pd.DataFrame(x, columns=['sample'])

df2['sample_base'] = df2['sample'] // 1
df2['sample_sqrt'] = np.sqrt(df2['sample'])
df2

Now we can add this new column into the original DataFrame by matching up the values on a shared column. In this case we want to match up on the original `sample` column.

Sometimes we have multiple DataFrames with one or more overlapping columns and we need to combine into a single DataFrame. This is where merging comes in.

Merging is a powerful and complex subject. I frequently find myself here: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html to lookup various functionalities.

In [ ]:
merged = pd.merge(df, df2, on='sample', suffixes=['_original', '_new'])
merged

If a column name appears in both DataFrames but is not the column that you are merging on, the strings defined in the `suffixes` argument will be appended to the end of the column names.

DataFrames can be written and read from files very easily. Many formats are supported but comma separated values (CSV) is the most commonly used in astronomy. The `read_csv` function can actually read a variety of text file formats by specifying the `delimiter` argument.

You can also load a CSV directly from a URL.

In [ ]:
merged.to_csv('sample.csv')

!cat sample.csv

from_csv = pd.read_csv('sample.csv', index_col=0)
from_csv

### Best practices

It's almost always better to avoid loops when working with Numpy arrays or Pandas DataFrames. If I'm working on a complicated problem and I'm unsure whether to use a loop or array/DataFrame operations I usually write it up in a loop first so that I can conceptualize the problem a little easier then convert later to remove as many loops as possible.

## Activity #1 

Let's see how much faster it is to work with Numpy arrays over Python lists.

In [ ]:
import time

# First we'll create a long list
length = 10000000  # must be an int
x = list(range(length))

# now lets loop over all of the elements and add one then divide by two
# we will also use the time package to time how long it takes
t1 = time.time()
for i in range(len(x)):
    x[i] = (x[i] + 1) / 2
t2 = time.time()

print("Updated {:d} elements in {:4.3f} seconds.".format(length, t2-t1))

1. Change the length of the array and keep track of how long the calculation takes as a function of that length.

1. Plot the time as a function of list length.

1. Now construct a Numpy array from the list `x` and perform the same calculation for several different array lengths.

1. (bonus) Plot the calculation time as a function of array length and add this line to the plot created in step #2.

## Activity #2

Lets load a couple files into a Pandas DataFrame and rearrange and merge them into a single file in a more useful format. `example_data/star_names.json` contains a list star names. The `primary_name` column is the primary ID for the star. For each unique `primary_name` there are many `other_names` associated with it. Each `primary_name`+`other_name` combination is stored in a separate row.

1. First load the file `example_data/star_names.json` into a Pandas DataFrame. The file is in JSON format so you might look into the `pandas.read_json` function.

1. Group the DataFrame on the `primary_name` column and create a custom aggregation function that takes all of the values in the `other_name` column that have the same `primary_name` and converts them into a single string deliminated with a pipe (`|`).

1. Load the `example_data/star_props.csv` file into a separate DataFrame and merge this with the grouped DataFrame from step #2.

1. Save the result as a new CSV file. The resulting file should look like `example_data/stars_merged.csv`. You may also load this file into Pandas to see what the final DataFrame should look like before saving to a CSV.